In [4]:
# *******UNIR************

In [5]:
# ************ Versión 29/01/2024 ***************
# ******** Creado por Javier Ulloa **************
# ************** TFM ****************************

In [1]:
# En este archivo se obtienen datos especificos de los discursos
import os
from datetime import datetime
from PyPDF2 import PdfReader
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy


##Codigo Adicional agregado
import sys

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from config import PDF_DIRECTORY_BOW, CSV_DIRECTORY_BOW


# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_lg") 

csv_output_path = CSV_DIRECTORY_BOW+"\general_information.csv"
ruta_carpeta = PDF_DIRECTORY_BOW



# Descargar las stopwords
nltk.download("stopwords")


# Crear o abrir el archivo CSV para escritura
with open(csv_output_path, "w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Escribir encabezados en el archivo CSV
    csv_writer.writerow(
        [
            "nombre_archivo_pdf",
            "país",
            "año",
            "nombre del presidente",
            "genero",
            "edad",
            "numero_de_paginas",
            "numero_de_palabras",
            "mayor_palabras_por_pagina",
            "menor_palabras_por_pagina",
            "promedio_palabras_por_pagina",
            "numero_adjetivos",
            "numero_sustantivos",
            "numero_verbos",
        ]
    )

    # Recorrer todos los archivos o discursos de la carpeta
    for nombre_archivo in os.listdir(ruta_carpeta):
        if nombre_archivo.endswith(".pdf"):
            ruta_pdf = os.path.join(ruta_carpeta, nombre_archivo)

            # Obtener información del nombre del archivo
            elementos = nombre_archivo.replace(".pdf", "").split("_")
            pais = elementos[0]
            año = int(elementos[1])
            nombre_presidente = " ".join(elementos[2:])

            # Inicializar variables para estadísticas
            num_pag = 0
            numero_palabras = 0
            mayor_palabras_por_pagina = 0
            menor_palabras_por_pagina = float("inf")
            total_palabras_por_pagina = 0

            # Inicializar variables para conteo de partes del discurso
            num_adjetivos = 0
            num_sustantivos = 0
            num_verbos = 0

            # Agregar columnas de genero y edad con valor inicial 'None'
            genero = None
            edad = None

            # Obtener estadísticas del PDF
            try:
                with open(ruta_pdf, "rb") as archivo_pdf:
                    pdf_reader = PdfReader(archivo_pdf)
                    num_pag = len(pdf_reader.pages)

                    # Calcular estadísticas de palabras por página
                    for pagina in pdf_reader.pages:
                        texto_pagina = pagina.extract_text()
                        palabras_pagina = word_tokenize(texto_pagina)
                        num_palabras_pagina = len(palabras_pagina)

                        total_palabras_por_pagina += num_palabras_pagina
                        mayor_palabras_por_pagina = max(
                            mayor_palabras_por_pagina, num_palabras_pagina
                        )
                        menor_palabras_por_pagina = min(
                            menor_palabras_por_pagina, num_palabras_pagina
                        )

                        # Procesar el texto con spaCy para obtener partes del discurso
                        doc = nlp(texto_pagina)
                        num_adjetivos += sum(1 for token in doc if token.pos_ == "ADJ")
                        num_sustantivos += sum(
                            1 for token in doc if token.pos_ == "NOUN"
                        )
                        num_verbos += sum(1 for token in doc if token.pos_ == "VERB")

                    # Calcular el promedio de palabras por página
                    promedio_palabras_por_pagina = (
                        total_palabras_por_pagina / num_pag if num_pag > 0 else 0
                    )

                # Obtener el número de palabras del contenido del PDF
                with open(ruta_pdf, "r", encoding="utf-8", errors="ignore") as archivo:
                    contenido = archivo.read()
                    numero_palabras = len(word_tokenize(contenido))

            except Exception as e:
                print(f"Error procesando {ruta_pdf}: {e}")

            # Escribir datos en el archivo CSV
            csv_writer.writerow(
                [
                    nombre_archivo,
                    pais,
                    año,
                    nombre_presidente,
                    genero,
                    edad,
                    num_pag,
                    numero_palabras,
                    mayor_palabras_por_pagina,
                    menor_palabras_por_pagina,
                    promedio_palabras_por_pagina,
                    num_adjetivos,
                    num_sustantivos,
                    num_verbos,
                ]
            )

# Imprimir mensaje de finalización
print("Fin del proceso, se genera el archivo CSV:", csv_output_path)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fin del proceso, se genera el archivo CSV: d:\TFM_Project\PLN_Project\data-BoW\csv\general_information.csv
